In [41]:
import torch
import einops

from circuit_lens import get_model_encoders
from z_sae import ZSAE
from mlp_transcoder import SparseTranscoder
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int
from torch import Tensor
from typing import List, Dict, TypedDict, Any, Union, Tuple, Optional
from tqdm import trange
from plotly_utils import imshow
from pprint import pprint
from transformer_lens.utils import get_act_name, to_numpy
from enum import Enum
from dataclasses import dataclass
from tqdm import tqdm 

# Import plotly stuff
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [2]:
def process_seq_index(tokens, seq_index):
    n_tokens = len(tokens)
    if seq_index < 0:
        seq_index += n_tokens
    return seq_index

In [3]:
model, z_saes, transcoders = get_model_encoders(device='cpu')

/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loaded pretrained model gpt2-small into HookedTransformer


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 12/12 [00:04<00:00,  2.62it/s]


In [4]:
len(z_saes)

12

In [5]:
prompt = "Forest is to tree as sky is to cloud"
tokens = model.to_tokens(prompt)
logits, cache = model.run_with_cache(tokens)

In [6]:
layer = 9
seq_index = -2
seq_index = process_seq_index(tokens, seq_index)

layer_z = einops.rearrange(
    cache["z", layer][0, seq_index],
    "n_heads d_head -> (n_heads d_head)",
)

print(layer_z.shape)

torch.Size([768])


In [7]:
_, z_recon, z_acts, _, _ = z_saes[layer](layer_z)

In [8]:
z_recon.shape, z_acts.shape

(torch.Size([768]), torch.Size([24576]))

In [9]:
sae_error = layer_z - z_recon
sae_error

tensor([ 7.0157e-02,  7.1703e-02,  2.3363e-01,  3.5861e-01,  2.0213e-01,
         1.3408e-01,  3.2548e-01,  2.3148e-01,  3.7720e-01, -2.5025e-01,
        -2.1057e-01, -2.3851e-01,  2.3197e-01,  2.4023e-01,  1.4382e-02,
        -3.3220e-02,  9.0975e-02,  2.8427e-01, -3.7215e-03,  1.1231e-01,
         2.2920e-01, -5.8852e-02,  1.5306e-01, -2.3493e-01, -1.9300e-01,
        -1.9187e-01, -2.0395e-01,  2.0610e-01,  2.9720e-02, -3.3919e-02,
         2.8115e-02,  2.1890e-01,  1.8478e-01, -7.7957e-02,  1.3496e-01,
         3.4570e-01, -1.1305e-01, -1.5915e-01,  8.9999e-03, -1.4639e-01,
         1.7077e-01,  4.3786e-02, -4.6952e-01,  2.3977e-01,  3.5124e-03,
         2.5602e-01,  1.2772e-01,  8.4909e-02,  2.6550e-01, -1.0964e-01,
        -1.3403e-01,  6.5226e-02,  1.6067e-01,  1.7311e-01,  1.1640e-01,
        -3.4142e-01, -7.5401e-02, -1.2113e-01, -4.0713e-02,  7.3968e-02,
        -2.1001e-01,  1.1112e-01, -1.1536e-01,  2.5154e-02, -8.0788e-02,
        -1.5961e-02, -1.0148e-01, -5.4164e-02, -1.8

In [10]:
px.line(to_numpy(sae_error))

In [11]:
from datasets import load_dataset

dataset = load_dataset("NeelNanda/pile-10k")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found cached dataset parquet (/Users/charlesoneill/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--pile-10k-72f566e9f7c464ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
# Split the huggingface dataset up into seq_len text
seq_len = 128
batch_size = 4096
model_name = 'gpt2-small'

model = HookedTransformer.from_pretrained(model_name, device='cpu')

tokenized_dataset = []
# Concat all the text together
text = " ".join(dataset['train']['text'])

# Tokenize the text
for i in trange(0, len(text), 2500):
    tokens = model.to_tokens(text[i:i+2500]).squeeze()
    # Split into seq_len chunks
    for j in range(0, len(tokens), seq_len):
        tokenized_dataset.append(tokens[j:j+seq_len])

Loaded pretrained model gpt2-small into HookedTransformer


100%|██████████| 24280/24280 [00:30<00:00, 786.65it/s]


In [49]:
# Keep only examples with seq_len 128
tokenized_dataset = [x for x in tokenized_dataset if len(x) == seq_len]

In [54]:
len(tokenized_dataset)

118061

In [72]:
# Assert all tensors have shape seq_length
for i, tokens in enumerate(tokenized_dataset):
    assert tokens.shape[0] == seq_len, f"Token {i} has shape {tokens.shape}"

In [84]:
# Turn tokenized_dataset (a list of tensors) into a Pytorch Dataset
from torch.utils.data import Dataset

batch_size = 64

class TokenizedDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.tokenized_dataset = tokenized_dataset

    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, idx):
        return self.tokenized_dataset[idx]
    
dataset = TokenizedDataset(tokenized_dataset)

from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print(next(iter(dataloader)).shape)

torch.Size([64, 128])


In [85]:
# Disable torch grad
torch.set_grad_enabled(False)

In [86]:
sae = z_saes[9]

# Get all z activations
z_acts = []
for batch in tqdm(dataloader):
    logits, cache = model.run_with_cache(batch)
    z = cache["z", layer] # batch_size x seq_len x n_heads x d_head
    del logits
    del cache
    z = einops.rearrange(
        z, 
        "b s n d -> (b s) (n d)"
    )
    z_acts.append(z)

# Stack all z activations along first dimension
z_acts = torch.cat(z_acts, dim=0)
z_acts.shape

  1%|          | 13/1845 [01:07<2:38:35,  5.19s/it]


KeyboardInterrupt: 

In [65]:
# Save tensors
torch.save(z_acts, "z_acts.pt")

In [66]:
# Load z_acts
z_acts = torch.load("z_acts.pt")

In [67]:
z_acts.shape

torch.Size([709632, 64])

In [68]:
# Create SAE dataset
class SAEDataset(Dataset):
    def __init__(self, z_acts):
        self.z_acts = z_acts

    def __len__(self):
        return len(self.z_acts)

    def __getitem__(self, idx):
        return self.z_acts[idx]
    
sae_dataset = SAEDataset(z_acts)

# Create SAE dataloader
sae_dataloader = DataLoader(sae_dataset, batch_size=batch_size, shuffle=True)

In [70]:
# Get SAE errors on each z_acts - we need to store the errors, and the original z_acts
sae_errors = []
for z in tqdm(z_acts):
    _, z_recon, z_acts, _, _ = sae(z)
    print(z.shape, z_recon.shape)
    sae_error = z - z_recon
    sae_errors.append(sae_error)
    break

  0%|          | 0/709632 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (64) must match the size of tensor b (768) at non-singleton dimension 0

In [33]:
z_saes[9].cfg

{'seed': 49,
 'batch_size': 4096,
 'buffer_mult': 384,
 'lr': 0.0012,
 'num_tokens': 2000000000,
 'l1_coeff': 1.2,
 'beta1': 0.9,
 'beta2': 0.99,
 'dict_mult': 32,
 'seq_len': 128,
 'enc_dtype': 'fp32',
 'model_name': 'gpt2-small',
 'site': 'z',
 'layer': 9,
 'device': 'cpu',
 'reinit': 'reinit',
 'head': 'cat',
 'concat_heads': True,
 'resample_scheme': 'anthropic',
 'anthropic_neuron_resample_scale': 0.2,
 'dead_direction_cutoff': 1e-06,
 're_init_every': 25000,
 'anthropic_resample_last': 12500,
 'resample_factor': 0.01,
 'num_resamples': 4,
 'wandb_project_name': 'gpt2-L9-20240117',
 'wandb_entity': 'ckkissane',
 'save_state_dict_every': 50000,
 'b_dec_init': 'zeros',
 'sched_type': 'cosine_warmup',
 'sched_epochs': 1000,
 'sched_lr_factor': 0.1,
 'sched_warmup_epochs': 1000,
 'sched_finish': True,
 'anthropic_resample_batches': 100,
 'eval_every': 1000,
 'model_batch_size': 512,
 'buffer_size': 1572864,
 'buffer_batches': 12288,
 'act_name': 'blocks.9.attn.hook_z',
 'act_size': 76